<a href="https://colab.research.google.com/github/gondow86/lstm_cnn_ensemble/blob/dev/LSTM_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [82]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [83]:
print(device)

cpu


In [84]:
# Hyper parameters
num_classes = 10
num_epochs = 2
batch_size = 100
learning_rate = 0.001

input_size = 28
sequence_length = 28
hidden_size = 128
num_layers = 1

In [85]:
from torchvision.transforms.transforms import ToTensor
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

In [86]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [87]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

    # out, _ = self.rnn(x, h0)
    out, _ = self.lstm(x, (h0, c0))

    out = out[:, -1, :]

    out = self.fc(out)

    return out

In [88]:
model = LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [89]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/`{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/`2], Step [100/600], Loss: 0.7683
Epoch [1/`2], Step [200/600], Loss: 0.5769
Epoch [1/`2], Step [300/600], Loss: 0.4399
Epoch [1/`2], Step [400/600], Loss: 0.3883
Epoch [1/`2], Step [500/600], Loss: 0.2502
Epoch [1/`2], Step [600/600], Loss: 0.2037
Epoch [2/`2], Step [100/600], Loss: 0.2353
Epoch [2/`2], Step [200/600], Loss: 0.1008
Epoch [2/`2], Step [300/600], Loss: 0.1920
Epoch [2/`2], Step [400/600], Loss: 0.1673
Epoch [2/`2], Step [500/600], Loss: 0.1971
Epoch [2/`2], Step [600/600], Loss: 0.1552


In [90]:
# from IPython.testing import test
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.reshape(-1, sequence_length, input_size).to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1) # softmax通してないけどクラス分の出力から最大値選べばOK
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')
    

Accuracy of the network on the 10000 test images: 97.0 %
Accuracy of the network on the 10000 test images: 97.5 %
Accuracy of the network on the 10000 test images: 97.0 %
Accuracy of the network on the 10000 test images: 96.0 %
Accuracy of the network on the 10000 test images: 95.2 %
Accuracy of the network on the 10000 test images: 95.33333333333333 %
Accuracy of the network on the 10000 test images: 95.14285714285714 %
Accuracy of the network on the 10000 test images: 95.125 %
Accuracy of the network on the 10000 test images: 95.11111111111111 %
Accuracy of the network on the 10000 test images: 94.9 %
Accuracy of the network on the 10000 test images: 94.63636363636364 %
Accuracy of the network on the 10000 test images: 94.58333333333333 %
Accuracy of the network on the 10000 test images: 94.38461538461539 %
Accuracy of the network on the 10000 test images: 94.5 %
Accuracy of the network on the 10000 test images: 94.46666666666667 %
Accuracy of the network on the 10000 test images: 94